# Measurement Error Mitigation

In [39]:
from qiskit import QuantumCircuit, QuantumRegister

import numpy as np

In [40]:
from qiskit_aer.noise import NoiseModel, pauli_error

#Simple noise model, which randomly flips each bit in an output with probability p.
def get_noise(p):
    error_meas = pauli_error([('X',p), ('I', 1 - p)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
        
    return noise_model

In [41]:
def initialize(n_qubits):
    total_states = 2**n_qubits
    states = [format(i, f'0{n_qubits}b') for i in range(total_states)]
    confusion_matrix = np.zeros((total_states, total_states))
    return states, confusion_matrix

In [47]:
from qiskit_aer.backends import AerSimulator
import scipy.linalg as la

shots = 100000
n_qubits = 2

states, confusion_matrix = initialize(n_qubits)

print(states)

for i, state in enumerate(states):
    qc = QuantumCircuit(n_qubits,n_qubits)
    if state[0]=='1':
        qc.x(1) # qiskit has inverted notation!!!!
    if state[1]=='1':
        qc.x(0)  
    qc.measure(qc.qregs[0],qc.cregs[0])  # [0] refers to the first register (to specify qubit, add [n])
    #print(qc)
    noise = get_noise(0.1)
    simulator = AerSimulator(noise_model = noise)
    counts = simulator.run(qc,shots=shots).result().get_counts(qc)
    print(state +' becomes',counts)

    # Update the confusion matrix
    for measured_state, count in counts.items():
        j = states.index(measured_state)
        confusion_matrix[i, j] += count

# Normalize the confusion matrix
confusion_matrix /= shots    


np.set_printoptions(formatter={'float': '{:0.6f}'.format})

print("\nConfusion Matrix:")
print(confusion_matrix, "\n")

mitigator = la.inv(confusion_matrix)

print("Mitigator:")
print(mitigator)  

#"""
#One could in theory find the confusion matrix and its inverse based on purely theoretical considerations of the error model, 
#or do it statistically as shown here. On hardware, probably the latter is better if the noise characterization isn't sufficiently accurate.
#"""

['00', '01', '10', '11']
00 becomes {'11': 1063, '01': 9084, '10': 9033, '00': 80820}
01 becomes {'10': 989, '00': 9192, '11': 9184, '01': 80635}
10 becomes {'01': 1003, '11': 8945, '00': 9107, '10': 80945}
11 becomes {'00': 1039, '10': 9206, '01': 9174, '11': 80581}

Confusion Matrix:
[[0.808200 0.090840 0.090330 0.010630]
 [0.091920 0.806350 0.009890 0.091840]
 [0.091070 0.010030 0.809450 0.089450]
 [0.010390 0.091740 0.092060 0.805810]] 

Mitigator:
[[1.269343 -0.142975 -0.141642 0.015273]
 [-0.144819 1.272761 0.017107 -0.145048]
 [-0.142834 0.016330 1.267142 -0.140638]
 [0.016439 -0.144924 -0.144886 1.273371]]


In [43]:
import scipy.optimize
import numpy.typing as npt

def closest_positive_distribution(
    quasi_probabilities: npt.NDArray[np.float64],
) -> npt.NDArray[np.float64]:
    """Given the input quasi-probability distribution returns the closest
    positive probability distribution (with respect to the total variation
    distance).

    Args:
        quasi_probabilities: The input array of real coefficients.

    Returns:
        The closest probability distribution.
    """
    quasi_probabilities = np.array(quasi_probabilities, dtype=np.float64)
    init_guess = quasi_probabilities.clip(min=0)
    init_guess /= np.sum(init_guess)

    def distance(probabilities: npt.NDArray[np.float64]) -> np.float64:
        return np.linalg.norm(probabilities - quasi_probabilities)

    num_vars = len(init_guess)
    bounds = scipy.optimize.Bounds(np.zeros(num_vars), np.ones(num_vars))
    normalization = scipy.optimize.LinearConstraint(np.ones(num_vars).T, 1, 1)
    result = scipy.optimize.minimize(
        distance,
        init_guess,
        bounds=bounds,
        constraints=normalization,
    )
    if not result.success:
        raise ValueError(
            "REM failed to determine the closest positive distribution."
        )
    return result.x

In [48]:
#now we define an arbitrary circuit and apply the mitigation 

shots = 100000

qc = QuantumCircuit(n_qubits,n_qubits)
qc.h(0)
qc.cx(0,1) 
qc.measure(qc.qregs[0],qc.cregs[0])
#print(qc)
simulator = AerSimulator(noise_model = noise) # Use same noise model as for calibration !!!
counts = simulator.run(qc,shots=shots).result().get_counts(qc)
# Normalize the counts to get probabilities
probabilities = {k: v / shots for k, v in counts.items()}

# Sort the probabilities dictionary by its keys (binary strings converted to integers)
probability_vector = np.array([probability for state, probability in sorted(probabilities.items(), key=lambda x: int(x[0], 2))])

print("Empirical probabilities:", probability_vector)

mitigated_probabilities = np.dot(mitigator, probability_vector)
print("Mitigated quasi-probabilities:", mitigated_probabilities)

closest_probabilities = closest_positive_distribution(mitigated_probabilities)
print("Closest positive probabilities:", closest_probabilities)

Empirical probabilities: [0.410000 0.092210 0.089900 0.407890]
Mitigated quasi-probabilities: [0.500743 0.000360 -0.000505 0.499747]
Closest positive probabilities: [0.500460 0.000077 0.000000 0.499464]


In [45]:
from typing import List
import numpy as np
import numpy.typing as npt

def sample_probability_vector(
    probability_vector: npt.NDArray[np.float64], samples: int
) -> List[List[int]]:
    """Generate a number of samples from a probability distribution as
    bitstrings.

    Args:
        probability_vector: A probability vector.
        samples: Number of samples to generate.

    Returns:
        A list of sampled bitstrings.
    """
    # sample using the probability distribution given
    num_values = len(probability_vector)
    choices = np.random.choice(num_values, size=samples, p=probability_vector)

    # convert samples to binary strings
    bit_width = int(np.log2(num_values))
    binary_repr_vec = np.vectorize(np.binary_repr)
    binary_strings = binary_repr_vec(choices, width=bit_width)

    # convert binary strings to lists of integers
    bitstrings = [list(map(int, list(bs))) for bs in binary_strings]

    return bitstrings


In [46]:
#if we are interested in generating some samples from the mitigated distribution
samples = 10
bitstrings = sample_probability_vector(closest_probabilities, samples)
print(bitstrings)

[[1, 1], [1, 1], [0, 0], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [0, 0], [1, 1]]
